In [38]:
import numpy as np
import pyamg
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression

from scipy.sparse.linalg import eigsh

import scipy
%matplotlib inline

In [2]:
grid_sizes = [ 5, 10,
              15, 20, 
              25, 30, 
              35, 40, 
              45, 50,
              55, 60,
              65, 70,
              75, 80,
              85, 90,
              95, 100] #20 total

scales = [1e-7, 5e-7,
          1e-6, 5e-6, 
          1e-5, 5e-5, 
          1e-4, 5e-4, 
          1e-3, 5e-3,]

runs_per = 10

def cg(A, b):
    return pyamg.krylov.cg(A, b, tol=1e-8, maxiter=min(A.shape[0],1000))[0]

def mg(A, b):
    return pyamg.ruge_stuben_solver(A).solve(b, tol=1e-8)

def sd(A, b):
    return pyamg.krylov.steepest_descent(A, b, tol=1e-8)[0]

def gmres(A, b):
    return pyamg.krylov.gmres(A, b, tol=1e-8, maxiter=min(A.shape[0],1000))[0]

methods = [("CG",cg), ("MG",mg), ("SD", sd), ("GMRES", gmres)]

In [3]:
results_df = pd.DataFrame(columns=['Method','N','sigma','run',
                                   'Error Norm','Residual Norm'])

for grid_i in range(len(grid_sizes)):
    temp_data = []
    print("Grid: ", grid_i)
    n = grid_sizes[grid_i]
    A = pyamg.gallery.poisson((n,n), format='csr')

    for scale_i in range(len(scales)):
        print("    Scale: ", scale_i)

        for run_i in range(runs_per):
            b = np.random.rand(A.shape[0])
            E = scipy.sparse.csr_matrix(np.random.normal(scale=scales[scale_i], size=(A.shape[0],A.shape[1]))).multiply(A!=0)
            Ap = A + E
            
            for method_i in range(len(methods)):
                name, solver = methods[method_i]
                x = solver(A, b)
                xp = solver(Ap, b)

                temp_data.append({
                    'Method':name,
                    'N':A.shape[0],
                    'sigma':scales[scale_i],
                    'run':run_i,
                    'Error Norm': np.linalg.norm(b - A@xp),
                    'Residual Norm': np.linalg.norm(x - xp)

                })
                
    results_df = results_df.append(temp_data)

NameError: name 'pd' is not defined

In [4]:
results_df.to_csv('Poisson_Perturbed_A_v2.csv')

NameError: name 'results_df' is not defined

In [55]:
method = 'SD'

features = ['N','sigma']
y = ['Error Norm']

method_results = results_df[(results_df['Method'] == method)].loc[:, features + y]
method_results['N^2*sigma'] = method_results['N']**2 * method_results['sigma']
features.append('N^2*sigma')

method_results['log({})'.format(y[0])] = np.log(method_results[y])
y = ['log(Error Norm)']

for feature in features:
    method_results['log({})'.format(feature)] = np.log(method_results[feature].astype(float))
#    MG_results['square({})'.format(feature)] = (MG_results[feature].astype(float))**2

 
features_to_fit = list(filter(lambda x: 'log' in x, method_results.columns))

reg = LinearRegression(normalize=True).fit(method_results[features_to_fit], method_results[y])
reg.score(method_results[features_to_fit], method_results[y]), reg.coef_

plt.figure(figsize=(10,10))
plt.scatter(method_results['log(N^2*sigma)'], method_results['log(Error Norm)'])
plt.xlabel('log(N^2*sigma)')
plt.ylabel('log(||x - x\'||')
plt.title('{}: Total Variance vs. Error Norm\nR^2: {}'.format(method,reg.score(method_results[features_to_fit], method_results[y])))
plt.show()

KeyError: 'log(N^2*sigma)'

<Figure size 720x720 with 0 Axes>